<a href="https://colab.research.google.com/github/Nandiniiy/Poetic-Texts-Project/blob/main/Generate_poetic_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
text = open(filepath, 'rb').read().decode(encoding = 'utf-8').lower()

In [ ]:
text = text[300000:800000]

In [ ]:
characters = sorted(set(text))

In [ ]:
char_to_index = dict((c, i) for i, c in enumerate(characters))

In [ ]:
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [ ]:
sentences= []
next_characters = []

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH , STEP_SIZE):
  sentences.append(text [i: i+SEQ_LENGTH])
  next_characters.append(text[i+SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)

In [ ]:
y = np.zeros((len(sentences), len(characters)), dtype=bool)

In [ ]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    x[i, t, char_to_index[character]] = 1
  y[i, char_to_index[next_characters[i]]] = 1

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.add(Dense(len(characters)))

In [ ]:
model.add(Activation('softmax'))

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))

In [ ]:
model.fit(x,y, batch_size=256, epochs=4)

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 117s 177ms/step - loss: 1.1589e-05
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 141s 176ms/step - loss: 8.8714e-06
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 145s 180ms/step - loss: 1.3602e-04
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 137s 173ms/step - loss: 4.9862e-05


In [ ]:
model.save('textgenerator.keras')

In [ ]:
model = tf.keras.models.load_model('textgenerator.keras')

In [ ]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [66]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    x = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      x[0, t, char_to_index[character]] = 1
      predictions = model.predict(x, verbose=0)[0]
      next_index = sample(predictions, temperature) # Pass predictions to sample function
      next_character = index_to_char[next_index]
      generated += next_character
      sentence = sentence[1:] + next_character
  return generated # Return outside of for loop

In [ ]:
print("-------0.2------")
print(generate_text(300, 0.2))
print("-------0.4------")
print(generate_text(300, 0.4))
print("-------0.6------")
print(generate_text(300, 0.6))
print("-------0.8------")
print(generate_text(300, 0.8))
print("-------1.0------")
print(generate_text(300, 1.0))

-------0.2------
as the author, thou the instrument.
therjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjvjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj